In [1]:
import keras 
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def baseline_model(nin, nh1 = 3000, nh2 = 1500, nh3 = 1000,nh4=500,nh5=250,nh6=100, nout = 47):
  
  # create model
    model = Sequential()
    model.add(Dense(nh1, input_shape=(nin,), activation='relu'))
    model.add(Dense(nh2, activation='relu'))
    model.add(Dense(nh3, activation='relu'))
    model.add(Dense(nh4, activation='relu'))
    model.add(Dense(nh6, activation='relu'))
    model.add(Dense(nout, activation='softmax'))
    opt = SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy','top_k_categorical_accuracy'])
    return model

In [25]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 100, 
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6,
              'max_leaf_nodes': None}
RF_model = RandomForestClassifier(**parameters)

In [3]:
import boto3

In [4]:
s3 = boto3.resource('s3')

In [5]:
content_object = s3.Object('mainfinaldata', '2019-05-17-06-46-59/a38703c1-dff8-488d-9647-1f148d0e6298')
file_content = content_object.get()['Body'].read().decode('utf-8')

In [6]:
import json

In [7]:
list_json = file_content.split('\n')

In [8]:
list_json[-1]

''

In [9]:
len(list_json)

20562

In [10]:
final_data = {}
global_data = set()
for i in list_json[:-1]:
    json_Data = json.loads(i)
    if len(json_Data['label-score']['m'])!=0:
        if json_Data['hash-tag']['s'] not in final_data:
            final_data[json_Data['hash-tag']['s']] = [json_Data['label-score']['m']]
        else:
            final_data[json_Data['hash-tag']['s']].append(json_Data['label-score']['m'])
        for label in json_Data['label-score']['m'].keys():
            global_data.add(label)

In [11]:
len(global_data)

4636

In [12]:
final_labels = final_data.keys()

In [13]:
final_data['#life']

[{'Cuisine': {'n': '0.944774448871612548828125'},
  'Blackberry': {'n': '0.96322524547576904296875'},
  'Cheesecake': {'n': '0.90234816074371337890625'},
  'Ingredient': {'n': '0.920731723308563232421875'},
  'Semifreddo': {'n': '0.888379275798797607421875'},
  'Berry': {'n': '0.950901925563812255859375'},
  'Dish': {'n': '0.993403494358062744140625'},
  'Dessert': {'n': '0.929272592067718505859375'},
  'Frozen dessert': {'n': '0.884287297725677490234375'},
  'Food': {'n': '0.990326106548309326171875'}},
 {'Cuisine': {'n': '0.902734458446502685546875'},
  'Strawberries': {'n': '0.64020812511444091796875'},
  'Ingredient': {'n': '0.823273241519927978515625'},
  'Recipe': {'n': '0.649404466152191162109375'},
  'Produce': {'n': '0.658344686031341552734375'},
  'Dish': {'n': '0.898547232151031494140625'},
  'Ice cream': {'n': '0.75092279911041259765625'},
  'Frozen dessert': {'n': '0.68687427043914794921875'},
  'Dessert': {'n': '0.613587200641632080078125'},
  'Food': {'n': '0.96686100959

In [14]:
# final_train_Data = [0]*len(global_data)
global_data = list(global_data)
global_data=sorted(global_data)
cleaned_data = {}
for key in final_data.keys():
    for image in final_data[key]:
#         print(images)
        encoded_image = [0.0]*len(global_data)
        for label in image.keys():
            encoded_image[global_data.index(label)] = float(image[label]['n'])
        if key not in cleaned_data:
            cleaned_data[key] = []
        cleaned_data[key].append(encoded_image)
#         break
#         for label in labels.keys():
#             final_train_Data[global_data.index(label)] = labels[label]        

In [15]:
global_data=sorted(global_data)

In [16]:
print(len(global_data))
print(global_data)

4636
['3d modeling', '3x3 (basketball)', 'A-line', 'Abaya', 'Abbey', 'Abdomen', 'Abseiling', 'Absolut vodka', 'Academic dress', 'Accessory fruit', 'Accipitridae', 'Accipitriformes', 'Accordion', 'Acerola', 'Acmon Blue', 'Acorn', 'Acoustic guitar', 'Acoustic-electric guitar', 'Acrobatics', 'Acrylic paint', 'Acting', 'Action figure', 'Active pants', 'Active tank', 'Actor', 'Acura', 'Acura tl', 'Adaptation', 'Adventure', 'Adventure game', 'Adventure racing', 'Advertising', 'Aegean cat', 'Aeolian landform', 'Aerial photography', 'Aerialist', 'Aerobics', 'Aerospace engineering', 'African elephant', 'African leopard', 'Africanis', 'Afro', 'Afterglow', 'Agama', 'Agaric', 'Agaricaceae', 'Agaricomycetes', 'Agaricus', 'Agave', 'Agent provocateur', 'Aggressive inline skating', 'Agriculture', 'Aguas frescas', 'Air gun', 'Air racing', 'Air show', 'Air sports', 'Air travel', 'Aircraft', 'Aircraft cabin', 'Airedale terrier', 'Airline', 'Airliner', 'Airplane', 'Airport', 'Airport apron', 'Aisle', 'Akb

In [17]:
dict = {'data':global_data}
# client = boto3.client('s3')
# # client.put_object(Body=csv_global_data, bucket)
with open('label_list.json', 'w') as file:
    json.dump(dict,file)
# client.upload_file('label_list.csv', 'mainfinaldata', 'label_list.csv')


In [18]:
final_labels = sorted(list(final_labels))
with open('cleaned_data.csv', 'w') as file:
    for k, vals in cleaned_data.items():
        for v in vals:
            v_str = [str(x) for x in v]
            v_str = ','.join(v_str)
#             print(v_str[:20])
            line = str(final_labels.index(k)) + ',' + v_str
            file.write(line + '\n')
#             print(line[:20])

In [19]:
client.upload_file('cleaned_data.csv', 'mainfinaldata', 'cleaned_data.csv')

NameError: name 'client' is not defined

In [3]:
import pandas as pd

In [3]:
data = pd.read_csv('cleaned_data.csv')

In [ ]:
data.head()

In [4]:
labels = data['24']

In [ ]:
len(labels)

In [26]:
del content_object

In [6]:
del data

In [5]:
df = data.drop(['24'],axis=1)

In [7]:
df.head()

,0.0,0.0.1,0.0.2,0.0.3,0.0.4,0.0.5,0.0.6,0.0.7,0.0.8,0.0.9,...,0.0.4616,0.0.4617,0.0.4618,0.0.4619,0.0.4620,0.0.4621,0.0.4622,0.0.4623,0.0.4624,0.0.4625
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
final_data = df.values

In [8]:
from sklearn.model_selection import train_test_split

In [11]:
del data
del final_data

NameError: name 'data' is not defined

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.values, labels.values, test_size=0.33, random_state=42)

In [18]:
import gc
gc.collect()
from keras.utils import np_utils

In [11]:
Y_train = np_utils.to_categorical(y_train, 45)

In [12]:
Y_test = np_utils.to_categorical(y_test, 45)

In [19]:
model = baseline_model(nin=X_train.shape[1],nout=45)

In [24]:
from sagemaker.sklearn import SKLearn
sklearn_estimator = SKLearn(entry_point='sklearn-train.py',
                            role='SageMakerRole',
                            train_instance_type='ml.m4.xlarge',
                            framework_version='0.20.0')

In [38]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 500,
              'max_features': 'sqrt'}
RF_model = RandomForestClassifier(**parameters)

In [ ]:
RF_model.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
RF_predictions = RF_model.predict(X_test)
score = accuracy_score(y_test ,RF_predictions)
print(score)

In [20]:
model.fit(X_train, Y_train, validation_split = 0.2,batch_size=32, epochs=100)

Instructions for updating:
Use tf.cast instead.
Train on 10444 samples, validate on 2612 samples
Epoch 1/100
10444/10444 [==============================] - 38s 4ms/step - loss: 3.7774 - acc: 0.0546 - top_k_categorical_accuracy: 0.2255 - val_loss: 3.7475 - val_acc: 0.0544 - val_top_k_categorical_accuracy: 0.2224
Epoch 2/100
10444/10444 [==============================] - 36s 3ms/step - loss: 3.7112 - acc: 0.0540 - top_k_categorical_accuracy: 0.2367 - val_loss: 3.6729 - val_acc: 0.0547 - val_top_k_categorical_accuracy: 0.2278
Epoch 3/100
10444/10444 [==============================] - 35s 3ms/step - loss: 3.6209 - acc: 0.0594 - top_k_categorical_accuracy: 0.2483 - val_loss: 3.5730 - val_acc: 0.0758 - val_top_k_categorical_accuracy: 0.2711
Epoch 4/100
10444/10444 [==============================] - 35s 3ms/step - loss: 3.5130 - acc: 0.0810 - top_k_categorical_accuracy: 0.3023 - val_loss: 3.4563 - val_acc: 0.1011 - val_top_k_categorical_accuracy: 0.3423
Epoch 5/100
10444/10444 [==============

KeyboardInterrupt: 

In [41]:
model.evaluate(X_test, Y_test, batch_size=32)
#y_pred = model.predict(X_test)

6431/6431 [==============================] - 21s 3ms/step


[2.323707883492578, 0.3417819934088897]

In [42]:
from keras.models import model_from_json

In [43]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [6]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

In [46]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [47]:
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [48]:
builder = builder.SavedModelBuilder(export_dir)

In [49]:
signature = predict_signature_def(
    inputs={"inputs": model.input}, outputs={"score": model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [50]:
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [53]:
!ls export

OSError: [Errno 12] Cannot allocate memory

In [52]:
del model

In [54]:
!ls export/Servo

OSError: [Errno 12] Cannot allocate memory

In [55]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [56]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [57]:
!touch train.py

OSError: [Errno 12] Cannot allocate memory

In [64]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [65]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

---------------------------------------------------------------------------------------!CPU times: user 576 ms, sys: 37.5 ms, total: 613 ms
Wall time: 7min 23s


In [70]:
endpoint_name = 'sagemaker-tensorflow-2019-01-29-17-36-55-837'

In [71]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

ClientError: An error occurred (ValidationException) when calling the DescribeEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:576327212885:endpoint/sagemaker-tensorflow-2019-01-29-17-36-55-837".

In [21]:
import json
import csv
import boto3
client = boto3.client('runtime.sagemaker')
# The sample model expects an input of shape [1,50]
source_list = {'Tree': 0.9838570952415466, 'Natural landscape': 0.9763058423995972, 'Nature': 0.9704620242118835, 'Natural environment': 0.926887571811676, 'Woodland': 0.9246786832809448, 'Grove': 0.9189432263374329, 'Nature reserve': 0.8917366862297058, 'Woody plant': 0.8800384402275085, 'Grass': 0.8748446106910706, 'Forest': 0.846611499786377}
global_list = []
with open('label_list.json') as file:
    data = json.load(file)
global_list = data['data']
#global_list = final_list
print(len(global_list))
final_label_list  = [0.0]*len(global_list)

for label in source_list.keys():
    if label in global_list:
        final_label_list[global_list.index(label)] = float(source_list[label])
data1 = [final_label_list]
data = np.random.randn(1, 4636).tolist()
print(len(data[0]))
print(len(data1[0]))
response = client.invoke_endpoint(EndpointName='sagemaker-tensorflow-2019-05-17-12-46-40-837', Body=json.dumps(data1))
response_body = response['Body']
prediction = json.loads(response_body.read())['outputs']['score']['floatVal']

4636
4636
4636


In [95]:
final_labels

NameError: name 'final_labels' is not defined

In [81]:
data

[[-1.3668620954805173,
  0.7240998678043798,
  -0.1010857286819474,
  -1.278580038191338,
  -0.41146170834687396,
  -0.36417752152077515,
  -0.16568168313615345,
  -0.593214631052193,
  1.0951664741289941,
  1.0863770953946084,
  -0.02350047029041348,
  0.5146481251738639,
  1.140556433805715,
  -0.0029665348144989933,
  0.43869007351669315,
  0.9081647069260615,
  1.1986556997219076,
  -1.780176780151785,
  0.797443091228848,
  2.2012211621413114,
  1.0431282532802455,
  -2.077909647598033,
  0.44648874660965165,
  -0.6358082834017149,
  0.8472964906476503,
  -0.6852015886243126,
  1.3594412941991185,
  0.3227029772527634,
  0.2969269585446106,
  -0.31443487734354986,
  -0.4927312418077028,
  0.8625304030358776,
  -0.10835723283199088,
  0.374879433551015,
  -0.2487837769690852,
  0.173039369773789,
  0.4679288667442785,
  0.06263221994297472,
  0.4315336336668134,
  0.7059563161673535,
  -0.2757166792817234,
  0.7865697587195588,
  -0.7310158075398931,
  -1.4627654589466548,
  0.3096

In [26]:
import sklearn as sk 

bt = xgb.XGBClassifier(max_depth=5,
                       learning_rate=0.2,
                       n_estimators=10,
                       objective='multi:softmax')   # Setup xgboost model
bt.fit(X_train, Y_train, # Train it to our data 
       verbose=False)

ModuleNotFoundError: No module named 'xgboost'

In [27]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 500,
              'max_features': 'sqrt'}
RF_model = RandomForestClassifier(**parameters)